In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

import warnings
warnings.filterwarnings('ignore')

print(requests.__version__)

2.32.3


In [4]:
DATA_PATH = '/content/drive/MyDrive/Colab Notebooks/iM_alphaco/data_set/'
with open(DATA_PATH + 'index.html', 'r', encoding='UTF8') as f:
    contents = f.read()                       # 이때까지만 해도 그냥 str 형식의 데이터이다!
    soup = BeautifulSoup(contents, 'lxml')    # 형변환 코드
    # print(soup)
    print(soup.h2)                            # 이 아래부터는 해당 태그를 가져오는 함수
    print(soup.ul)
    print(soup.ul.li)

    companies = []
    for tag in soup.find_all('li'):
        companies.append(tag.text)
    print(companies[-1])

<h2>모바일 컴퍼니</h2>
<ul id="mylist" style="width:150px">
<li>애플</li>
<li>삼성</li>
<li>노키아</li>
<li>LG</li>
</ul>
<li>애플</li>
LG


In [11]:
company_code = '139130' # DGB 금융지주
url ="https://finance.naver.com/item/sise_day.nhn?code=" + company_code

headers = {
         'referer' : 'https://finance.naver.com/item/sise.naver?code=139130',
         'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
        }

def crawling(soup):
    last_page = int(soup.select_one('td.pgRR').a['href'].split('=')[-1])
    # print(last_page)
    df = None
    count = 0
    for page in range(1, last_page + 1):
      req = requests.get(f'{url}&page={page}', headers=headers)
      df = pd.concat([df, pd.read_html(req.text, encoding = "euc-kr")[0]], ignore_index=True)
      if count > 10:
        break
      count += 1

    df.dropna(inplace=True)
    df.reset_index(drop=True, inplace=True)

    return df

def main():
    company_code = '139130' # 삼성전자
    url ="https://finance.naver.com/item/sise_day.nhn?code=" + company_code

    headers = {
             'referer' : 'https://finance.naver.com/item/sise.naver?code=139130',
             'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
            }
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    result = crawling(soup)
    print(result)

if __name__ == "__main__":
    main()

             날짜      종가      전일비      시가      고가      저가       거래량
0    2024.10.10  8140.0   상승  80  8130.0  8220.0  8080.0  476689.0
1    2024.10.08  8060.0  하락  130  8180.0  8180.0  8060.0  448627.0
2    2024.10.07  8190.0  상승  120  8080.0  8250.0  8010.0  449215.0
3    2024.10.04  8070.0   하락  20  8130.0  8130.0  8010.0  230492.0
4    2024.10.02  8090.0      보합0  8090.0  8150.0  8000.0  456282.0
..          ...     ...      ...     ...     ...     ...       ...
115  2024.04.17  8140.0      보합0  8170.0  8210.0  8070.0  315207.0
116  2024.04.16  8140.0   하락  10  8150.0  8220.0  8060.0  387470.0
117  2024.04.15  8150.0  상승  100  8030.0  8150.0  8010.0  362279.0
118  2024.04.12  8050.0   하락  90  8070.0  8130.0  8000.0  574946.0
119  2024.04.11  8140.0  하락  170  8160.0  8230.0  8060.0  628447.0

[120 rows x 7 columns]
